## <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea">Assignment 5: Credit Card Users Churn Prediction</span>

&nbsp;

<p align="center" style="font-family: Arial;color:#0e92ea;font-size:1em;">
The Thera bank recently saw a steep decline in the number of users of their credit card, credit cards are a good source of income for banks because of different kinds of fees charged by the banks like annual fees, balance transfer fees, and cash advance fees, late payment fees, foreign transaction fees, and others. Some fees are charged to every user irrespective of usage, while others are charged under specified circumstances. Goal is to come up with a classification model that will help the bank improve its services so that customers do not renounce their credit cards
</p>

##  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> Contents:</span>

<ol style="font-family: Arial;color:#0e92ea;font-size:1em;">
    <li>Exploratory Data Analysis and Insights</li>
    <li>Data pre-processing</li>
    <li>Model building</li>
    <li>Model building - Oversampled data</li>
    <li>Model building - Undersampled data</li>
    <li>Hyperparameter tuning using random search</li>
    <li>Model Performances</li>
    <li>Productionize the model</li>
    <li>Recommendations</li>
</ol>

In [ ]:
'''
pip install category_encoders
pip install xgboost
'''

In [ ]:
# To help with reading and manipulation of data
import numpy as np
import pandas as pd
import pprint

# To help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

# To split the data
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

# to create k folds of data and get cross validation score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# to create pipeline and make_pipeline
from sklearn.pipeline import Pipeline, make_pipeline

# to use standard scaler
from sklearn.preprocessing import StandardScaler

# To impute missing values
from sklearn.impute import SimpleImputer

# To build a Random forest classifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# To tune a model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

# To get different performance metrics
import sklearn.metrics as metrics
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    recall_score,
    accuracy_score,
    precision_score,
    f1_score,
)

from sklearn.base import BaseEstimator

# To undersample and oversample the data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To suppress warnings
import warnings

warnings.filterwarnings("ignore")
base_data = pd.read_csv('BankChurners.csv')

default_color_palette  = ["#03A9F4","#7C4DFF", "#FF5252", "#D50000", "#FF6F00", "#0288D1", "#7C4DFF"]
maritalstatus_color_palette = {'Single':'#ff3d00', 'Married':'#00c853', 'Divorced': '#EA112F'}
stats_colors           = {'Mean':'#D50000', 'Mode':'#FF3D00', 'Median':'#2962FF'} # Set standard colors for mean, mode and median to use accross entire notebook.
gender_color_palette   = {"F":"#E91E63", "M":"#42A5F5"} # Gender based colors.

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 1 and 2: Exploratory Data Analysis and Insights and Data Processing

In [ ]:
'''
Input:
Pandas DataFrame

Output:
Displays DataFrame structure 
(columns, nulls and non nulls counts and percentage highlighing columns with most number of nulls)

Retunrs:
N/A
'''
def info(dataFrame):
    print(f"{dataFrame.shape[0]} Rows x {dataFrame.shape[1]} Columns")
    nulls_series      = dataFrame.isna().sum()         # Get a series counting number of empty values for each column
    nonnulls_series   = dataFrame.notnull().sum()      # Get a series counting number of non empty valuesfor each column 
    nulls_percentage  = ((nulls_series * 100)/(nulls_series + nonnulls_series)).astype(float)
    column_datatypes  = dataFrame.dtypes               # Get a series containing data types for each column 
    
    series_arr    = [nulls_series, nonnulls_series, nulls_percentage, column_datatypes]
    col_names_arr = ["Nulls", "Non-Nulls","Nulls %", "Type"]
    
    nulls_count_df = pd.concat(
        objs = series_arr,
        axis = 1,
        keys = col_names_arr, 
        sort = True)
    
    cm = sns.light_palette("red", as_cmap=True)
    display(nulls_count_df.style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, ['Nulls %']]).format(formatter={('Nulls %'): "{:.2f}%"})) 
    

'''
Input:
Target type and a list of feature names.

Output:
Convert all features provided in 'column_names' to Target type provided in 'toType'

Returns:
modifies main original data frame and returns nothing.
'''
def ConvertColTo(toType = "category", column_names=np.nan, df = np.nan):
    for col_name in column_names:
        df[col_name] = df[col_name].astype(toType)


'''
Input:
N/A

Output:
Go through each categorial column and print unique values for that column.

Retunrs:
N/A
'''
def CountUniqueValues(col_names):
    for col_name in col_names:
        print(f"======================='{col_name}'==================")
        for unique_col_value in  main_data[col_name].unique().tolist():
            total_count = main_data[col_name].count()
            unique_values_count = main_data[main_data[col_name] == unique_col_value][col_name].count()
            percentage = str(round((unique_values_count/total_count) * 100, 2))
            print(f"{unique_col_value} \t: {unique_values_count} ({percentage}%)")
        print(f"=========================================================\n")
        
'''
Input:
list of column names

Output:
a dictionary contating coloumn with unique value counts
split by thhe threshold

Retunrs:
N/A
'''
def SplitColumnsByUniqueValueCountThreshold(col_names, threshold = 30):
    results = {
        'lower_set' : [],
        'upper_set': []
    }
    
    for col_name in col_names:            
        if len(list(main_data[col_name].unique())) < threshold:
            results['lower_set'].append(col_name)
        else:
            results['upper_set'].append(col_name)
            
    pprint.pprint(results)
    return results

'''
Description:
Given the model the test data y values, displays the confusion matrix for that model.

Input:
model    - The learning model.
y_actual - Y values from test data.
labels   - the labels

Returns:
N/A
'''
def DisplayConfusionMatrix(model, y_actual, labels=[1,0]):
    y_predict             = model.predict(x_test)
    confusion_matrix      = metrics.confusion_matrix(y_actual, y_predict, labels=[0, 1])
    
    confusion_matix_df = pd.DataFrame(
        data    = confusion_matrix,
        index   = [i for i in ["Actual No", "Actual Yes"]],
        columns = [i for i in ["Predicted - No", "Predicted - Yes"]])
    
    group_counts     = ["{0:0.0f}".format(value) for value in confusion_matrix.flatten()]
    group_percetages = ["{0:.2%}".format(value) for value in confusion_matrix.flatten()/np.sum(confusion_matrix)]
    
    labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts, group_percetages)]
    labels = np.array(labels).reshape(2, 2)
    plt.figure(figsize= (10,7))
    sns.heatmap(confusion_matix_df, annot=labels,fmt='')
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")

'''
Description:
Given the model, computes models perfomance on test and training data.
Perf metrics displayed are: Accuracy, Recall, F1 Score, and precision.

Input:
model - The learning model.

Returns:
A dictionary containing models perfomace.
'''
def GetMetricsScore(model, x_train_arg, x_test_arg, y_train_arg, y_test_arg):
    
    pred_train = model.predict(x_train_arg)
    pred_test  = model.predict(x_test_arg)
    
    train_accuracy = model.score(x_train_arg, y_train_arg)
    test_accuracy  = model.score(x_test_arg, y_test_arg)
    
    train_recall = metrics.recall_score(y_train_arg, pred_train)
    test_recall  = metrics.recall_score(y_test_arg, pred_test)
    
    train_precision = metrics.precision_score(y_train_arg, pred_train)
    test_precision  = metrics.precision_score(y_test_arg, pred_test)

    f1_score_train = 2 * ((train_precision * train_recall)/(train_precision + train_recall))
    f1_score_test = 2 * ((test_precision * test_recall)/(test_precision + test_recall))
    
    return {
        'Accuracy_Test'   : test_accuracy,
        'Accuracy_Train'  : train_accuracy,
        'Recall_Test'     : test_recall,
        'Recall_Train'    : train_recall,
        'Precision_Test'  : test_precision,
        'Precision_Train' : train_precision,
        'F1_Score_Train'  : f1_score_train,
        'F1_Score_Test'   : f1_score_test
    }
        
'''
Description:
Given the model, displays the importance score.

Input:
model - The learning model.

Returns:
N/A
'''
def DisplayImportanceScores(model):
    gini_importances = pd.DataFrame(
        data    = model.feature_importances_, 
        columns = ["Imp"],
        index   = x_train.columns).sort_values(by = "Imp")
    
    print(gini_importances)

'''
Description:
Given the model, displays the importance chart.

Input:
model    - The learning model.

Returns:
N/A
'''
def DisplayImportanceChart(model):
    importances = model.feature_importances_
    indices     = np.argsort(importances)

    plt.figure(figsize=(12, 12))
    plt.title('Feature Importance')
    plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
    plt.yticks(range(len(indices)), [list(X.columns)[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

'''
Description:
Given the decision tree model, displays the decision tree of that model.

Input:
model - The learning model.
size  - Size of the plot

Returns:
N/A
'''
def PlotDecisionTree(model, size= [20, 30]):
    plt.figure(figsize=(size[0],size[1]))
    tree.plot_tree(
        model, 
        feature_names=feature_names,
        filled=True,
        fontsize=9,
        node_ids=True,
        class_names=True)
    plt.show()
 
'''
Description:
Given a model, displays the coefient values of that model

Input:
model - The learning model.

Returns:
N/A
'''
def DisplayCoeficients(model):
    coefs = pd.DataFrame(
        np.append(model.coef_, model.intercept_),
        index=x_train.columns.tolist() + ["Intercept"],
        columns=["Coefficients"],
    )

    coefs.sort_values('Coefficients')

### Data Exploration

In [ ]:
base_data.head(10)

In [ ]:
TARGET_COLUMN = 'Attrition_Flag'

In [ ]:
info(base_data)

In [ ]:
base_data.drop("CLIENTNUM", axis=1, inplace=True)

In [ ]:
replace_struct = {
    "Attrition_Flag" : {"Existing Customer": 0, "Attrited Customer": 1},
}
base_data = base_data.replace(replace_struct)
base_data[TARGET_COLUMN] = pd.to_numeric(base_data[TARGET_COLUMN], downcast="integer")

In [ ]:
# Separating data into X and Y
X = base_data.drop([TARGET_COLUMN], axis = 1)
Y = base_data[TARGET_COLUMN]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=1)

print(x_train.shape, x_test.shape)

In [ ]:
main_data = pd.concat([x_train, y_train], axis=1)

In [ ]:
main_data.head(10)

### Missing Values Treatment

In [ ]:
catgry_col_names = main_data.select_dtypes(include=['object']).columns.tolist()

In [ ]:
ConvertColTo('category', catgry_col_names, main_data)
info(main_data)

In [ ]:
catgry_col_names = main_data.select_dtypes(include=['category']).columns.tolist()
CountUniqueValues(catgry_col_names)

In [ ]:
def MissingValuesTreatment(df):
    df['Education_Level'].fillna(df['Education_Level'].value_counts().idxmax(), inplace=True)
    df['Marital_Status'].fillna(df['Marital_Status'].value_counts().idxmax(), inplace=True)
    
MissingValuesTreatment(main_data)

In [ ]:
info(main_data)

In [ ]:
CountUniqueValues(catgry_col_names)

In [ ]:
        
'''
Description:
Displays a grid catplots

Input:
A list of column names
'''
def DisplayCountPlotGrid(col_names, hue_name, color_palette):
    col_index = 0
    for r in range(0, int(len(col_names)), 3):
        fig, axs = plt.subplots(
            nrows=1,                                                                # Number of rows of the grid
            ncols=3,                                                                # Number of columns of the grid.
            figsize=(15,4),
            constrained_layout=True)

        for index in range(0, 3):
            if col_index < int(len(col_names)):
                column_name = col_names[col_index]
                ax = axs.flat[index]
                ax = sns.countplot(
                    data=main_data,
                    x=main_data[column_name],
                    palette=color_palette,
                    hue=hue_name,
                    ax = ax)
                ax.set_xlabel(column_name)                                           
                ax.set_title(column_name + ' and '+ hue_name +' Profile', fontsize=14)
                if int(len(ax.get_xticklabels())) > 14: 
                    ax.set_xticklabels([], rotation=45, ha='right')
                else:
                    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
            col_index += 1
            
'''
Input:
N/A

Output:
Plot box plots in line.

Returns:
N/A
'''     
def PlotLineOfCountPlots(col_names, hue_name, palette_name):
    fig, axs = plt.subplots(
        len(col_names),
        figsize = (15,6),
        sharex  = False,
        sharey  = False)
    
    fig.subplots_adjust(top = 4)
    
    for i in range(len(col_names)):
        column_name = col_names[i]
        sns.countplot(
            data    = main_data, 
            x       = column_name,
            hue     = hue_name,
            palette = palette_name,
            ax      = axs[i])
        
        axs[i].set_xlabel(column_name)                                           
        axs[i].set_title(column_name + ' Profile', fontsize=14)
        
        if int(len(axs[i].get_xticklabels())) > 30: 
            axs[i].set_xticklabels([], rotation=45, ha='right')
        else:
            axs[i].set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

In [ ]:
cols_dic = SplitColumnsByUniqueValueCountThreshold(
    main_data.select_dtypes(include=['float', 'int64']).columns.tolist(),
    40)
descrete_data_columns        = cols_dic['lower_set']
large_descrete_data_columns  = cols_dic['upper_set']

In [ ]:
large_descrete_data_columns

In [ ]:
descrete_data_columns

### Exploring Univariate and Bivariate Analysis based on Gender
- The number of clients is higher in customers who are Graduates
- The number of clients is higher in customers who are Married and Single
- The number of clients is higher in customers who earn less that $40k per year
- Blue card has the highest number of clients and seems to be the one brining in more money

In [ ]:
catgry_col_names.append(TARGET_COLUMN)
print(catgry_col_names)

In [ ]:
DisplayCountPlotGrid(
    col_names     = catgry_col_names,
    hue_name      = "Gender",
    color_palette = gender_color_palette)

In [ ]:
DisplayCountPlotGrid(
    col_names    = descrete_data_columns,
    hue_name     = "Gender",
    color_palette = gender_color_palette)

### Exploring Univariate and Bivariate Analysis based on  Marital Status
- The charts sho a higher number of clients who are married and single
- Also the ratio of clients who left the service based on marital statis seems insignificance. Cannot deduce from the current of the data who category is most likely to stay or leave.
- Another noticeable difference is how dominant maried category is.
- Also most of the customers seems to be having a 2-3 dependents 

In [ ]:
CountUniqueValues(['Marital_Status'])

In [ ]:
DisplayCountPlotGrid(
    col_names    = catgry_col_names,
    hue_name     = "Marital_Status",
    color_palette = maritalstatus_color_palette)

In [ ]:
DisplayCountPlotGrid(
    col_names       = descrete_data_columns,
    hue_name        = "Marital_Status",
    color_palette   = maritalstatus_color_palette)

### Exploring Univariate and Bivariate Analysis based on Education
- Not that much noticeable differen except that most of the customers are graduates and they are also the ones most likely to leave the service

In [ ]:
CountUniqueValues(['Education_Level'])

In [ ]:
DisplayCountPlotGrid(
    col_names    = catgry_col_names,
    hue_name     = "Education_Level",
    color_palette = default_color_palette)

In [ ]:
DisplayCountPlotGrid(
    col_names       = descrete_data_columns,
    hue_name        = "Education_Level",
    color_palette   = default_color_palette)

### Exploring Univariate and Bivariate Analysis based on Card Type
- Again we see blue as the most dominant card in all categories.
- This blue card is also the one with the highest number of attrited customers.
- There is also a higher number of customers using the blue card who are earning less than $40k per year.

In [ ]:
CountUniqueValues(['Card_Category'])

In [ ]:
DisplayCountPlotGrid(
    col_names    = catgry_col_names,
    hue_name     = "Card_Category",
    color_palette = default_color_palette)

In [ ]:
DisplayCountPlotGrid(
    col_names       = descrete_data_columns,
    hue_name        = "Card_Category",
    color_palette   = default_color_palette)

### Exploring Univariate and Bivariate Analysis based on Income
- We notice a spike or peak values in customers who are earning less than \$40k per year
- The number of people leaving the credit card service based on income seems level or average accross all categories except for customers who are earning less than \$40k per year with the highest number of attrition
- Customers who earn less than \$40k per year also have 2-3 dependants

In [ ]:
CountUniqueValues(['Income_Category'])

In [ ]:
DisplayCountPlotGrid(
    col_names    = catgry_col_names,
    hue_name     = "Income_Category",
    color_palette = default_color_palette)

In [ ]:
DisplayCountPlotGrid(
    col_names       = descrete_data_columns,
    hue_name        = "Income_Category",
    color_palette   = default_color_palette)

### Data Processing and Feature engineering

In [ ]:
def ColumnTransformation(df):
    replace_struct = {
        "Card_Category"       : {"Blue": 1, "Silver": 2, "good": 3, "Gold": 4, "Platinum": 5},
        "Income_Category"     : {"abc": -1, "Less than $40K": 1, "$40K - $60K": 2, "$60K - $80K": 3, "$80K - $120K": 4, "$120K +": 5},
        "Education_Level"     : {"Uneducated": 1, "High School": 2, "College": 3, "Graduate": 4, "Post-Graduate":5, "Doctorate": 6},
        "Attrition_Flag"      : {"Existing Customer": 0, "Attrited Customer": 1},
    }

    df = df.replace(replace_struct)
    df.head(10)
    return df
    
main_data = ColumnTransformation(main_data)

In [ ]:
def OneHotEncoding(df):
    oneHotCols = ["Gender", "Marital_Status"]
    df = pd.get_dummies(df, columns=oneHotCols)
    info(df)
    return df

main_data = OneHotEncoding(main_data)

In [ ]:
def FixTargetFeatureType(df):
    df[TARGET_COLUMN] = pd.to_numeric(df[TARGET_COLUMN], downcast="integer")
    return df
    
main_data = FixTargetFeatureType(main_data)

In [ ]:
info(main_data)

### Corrolation and Skewness

In [ ]:
fig, ax = plt.subplots(figsize=(18,18)) 
sns.heatmap(data=main_data.corr().sort_values(by=[TARGET_COLUMN]), annot=True, linewidths=.5, ax=ax)
plt.show()

In [ ]:
'''
Input:
Axis, column name/x-axis, hue name

Output:
Displays a count plot.

Retunrs:
N/A
''' 
def HistBoxplot(box_chart_ax, hist_chart_ax, x_axis):
    sns.boxplot(
        data=main_data,
        x=main_data[x_axis],
        showmeans=True,
        ax=box_chart_ax)
    
    sns.histplot(
        data=main_data,
        x=main_data[x_axis],
        kde=True,
        ax=hist_chart_ax)
    
    hist_chart_ax.axvline(main_data[x_axis].mean(),                 # Get the mean of the values in the given column and draw a vertical line that cuts the chart on the mean value 
               color=stats_colors['Mean'],                                  # Use on of the colors predefined on this notebook
               label='Mean',                                                # Set the label to be diplayed on the legend
               linestyle="dashed");                                         # Make the line have dashes
    
    hist_chart_ax.axvline(main_data[x_axis].median(),               # Plot the median line on the chart.
               color=stats_colors['Median'],                                # Use on of the colors predefined on this notebook
               label='Median',                                              # Set the label to be diplayed on the legend
               linestyle="dashed");                                         # Make the line have dashes
    
    hist_chart_ax.axvline(main_data[x_axis].mode()[0],              # Plot the mode line on the chart.
               color=stats_colors['Mode'],                                  # Use on of the colors predefined on this notebook 
               label='Mode',                                                # Set the label to be diplayed on the legend
               linestyle="dashed");                                         # Make the line have dashes
    
    hist_chart_ax.legend(loc='upper right')

'''
Input:
N/A

Output:
Displays a a grid of [Boxplot x Distribution chart] for discrete features.

Returns:
N/A
''' 
def PlotHistBoxGrid():
    col_names = main_data.select_dtypes(include=['float', 'int64']).columns.tolist()
    print(col_names)

    col_index = 0
    for r in range(0, int(len(col_names)), 3):
        fig, (box, hist) = plt.subplots(
            nrows=2,                                                                # Number of rows of the grid
            ncols=3,                                                                # Number of columns of the grid.
            figsize=(15,4),
            gridspec_kw={"height_ratios" : (0.25,0.5)},
            constrained_layout=True)

        for index in range(0, 3):
            if col_index < int(len(col_names)):
                HistBoxplot(box.flat[index], hist.flat[index], col_names[col_index])
            col_index += 1

In [ ]:
'''
Input:
Column name

Output:
A series containing interquatile range values

Retunrs:
A dictionary containing quatile range values
''' 
def Get_IQR(data):
    quartiles = np.quantile(data, [.25, .75])
    iqr = (quartiles[1] - quartiles[0])
    #print(f'Q1 = {quartiles[0]}, Q3 = {quartiles[1]}, IQR = {iqr}')
    return {
        "Q1": quartiles[0],
        "Q3": quartiles[1],
        "IQR": iqr
    }

'''
Description:
replace outlier value with a given value (replacement_value) and an IQR multiplier (scale)

Input:
x                  - value to replace
replacement_value  - value to replace the outlier with.
quatiles           - dictionary containing quatile ranges {"Q1": X, "Q2": Y, "IQR": Z}
scale.             - IQR multiplier

Returns:
new outlier value
'''
def ReplaceOutlier(x, replacement_value, quatiles, scale):
    if x < quatiles["Q1"] - scale * quatiles["IQR"]  or x > quatiles["Q3"] + scale * quatiles["IQR"]:
        return replacement_value
    return x

'''
Description:
Iterates through the IQR multiplier, to find the multiplier that
yeild best corrolation with target variable
'''
def BestCorrOutlierTreatment(feature_column, target, df):
    max_corr = 0.0
    best_feature = df[feature_column]

    for r in range(10):
        main_data_copy = df.copy()
        main_data_copy[feature_column] = main_data_copy[feature_column].apply(
            ReplaceOutlier,
            convert_dtype     = True,
            replacement_value = main_data_copy[feature_column].median(),
            scale             = r, 
            quatiles          = Get_IQR(main_data_copy[feature_column]))
        
        current_corr = main_data_copy.corr()[feature_column][target]
         # print(f"IQR Scale: {r}, Scale Corrolation : {current_corr}")
        
        if current_corr > max_corr:
            max_corr = current_corr
            best_feature = main_data_copy[feature_column]
            
    for r in range(10):
        main_data_copy = df.copy()
        main_data_copy[feature_column] = main_data_copy[feature_column].apply(
            ReplaceOutlier,
            convert_dtype     = True,
            replacement_value = main_data_copy[feature_column].mean(),
            scale             = r, 
            quatiles          = Get_IQR(main_data_copy[feature_column]))
        
        current_corr = main_data_copy.corr()[feature_column][target]
        
        if current_corr > max_corr:
            max_corr = current_corr
            best_feature = main_data_copy[feature_column]
            
    return best_feature

'''
Input:
N/A

Output:
Plot box plots in line.

Returns:
N/A
''' 
def PlotBoxes():
    columns = main_data.select_dtypes(include=['float', 'int64']).columns.tolist()
    fig, axs = plt.subplots(len(columns), figsize=(15,10), sharex=False, sharey=False)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=10, wspace=10, hspace=None)
    for i in range(len(columns)):
        column_name = columns[i]
        sns.boxplot(data=main_data[column_name], orient="h", palette="Set2", ax = axs[i])
        axs[i].set_xlabel(column_name)                                           
        axs[i].set_title(column_name + ' Profile', fontsize=14)

### Outlier Treatment

In [ ]:
def TreatUoutliers(df):
    columns = main_data.select_dtypes(include=['float', 'int64']).columns.tolist()

    main_data_copy = df.copy()
    for column in columns:
        main_data_copy[column] = BestCorrOutlierTreatment(column, TARGET_COLUMN, main_data_copy)
    
    return main_data_copy

In [ ]:
df1 = main_data.corr()[[TARGET_COLUMN]].copy()
df1["Tranformation"] = "Before"

df2 = TreatUoutliers(main_data).corr()[[TARGET_COLUMN]].copy()
df2["Tranformation"] = "After"

df = pd.concat([df1, df2], axis=0).reset_index()
df

In [ ]:
fig, ax = plt.subplots(figsize=(18, 12))

ax = sns.lineplot(data=df, x="index", y=TARGET_COLUMN, hue="Tranformation", ax=ax)
ax.set(title='Changes in corrolation value after Outlier Treatment')
ax.set(xlabel=None)
plt.xticks(rotation = 45) # Rotates X-Axis Ticks by 45-degrees
plt.show()

In [ ]:
main_data = TreatUoutliers(main_data)

### Pair Plot and Dropping Variables with high Variance and Very low corrolation

In [ ]:
# drop features with corrolation of less than 0.05 to the tagert variable
list_of_features_to_drop = main_data.corr()[abs(main_data.corr()[TARGET_COLUMN]) < 0.05].index.to_list()
main_data.drop(list_of_features_to_drop, axis=1, inplace=True)
info(main_data)

In [ ]:
main_data.head(10)

In [ ]:
sns.pairplot(main_data, kind="reg", hue=TARGET_COLUMN)

In [ ]:
fig, ax = plt.subplots(figsize=(18,18)) 
sns.heatmap(data=main_data.corr().sort_values(by=[TARGET_COLUMN]), annot=True, linewidths=.5, ax=ax)
plt.show()

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 4. Model Building and Evaluations

In [ ]:
X_2 = main_data.drop([TARGET_COLUMN], axis = 1)
Y_2 = main_data[TARGET_COLUMN]

x_train, x_val, y_train, y_val = train_test_split(X_2, Y_2, test_size=0.5, random_state=1)

print(x_train.shape, x_test.shape)

### Building Models with default values
- The RandomForestClaffier model perfoms better (recall and F1 score) than all other models in most of the labs ran.
- We select the RandomForest model to create the pipeline.

In [ ]:
def BuildAndGetModels(x_var, y_var):
    bagging_estimator=BaggingClassifier(random_state=1)
    bagging_estimator.fit(x_var, y_var)
    pprint.pprint(bagging_estimator)
    
    rf_estimator=RandomForestClassifier(random_state=1)
    rf_estimator.fit(x_var, y_var)
    pprint.pprint(rf_estimator)

    logistic_reg_model = LogisticRegression()
    logistic_reg_model.fit(x_var, y_var)
    pprint.pprint(logistic_reg_model)

    dtree1 = DecisionTreeClassifier(random_state=1)
    dtree1.fit(x_var, y_var)
    pprint.pprint(dtree1)

    adaBoosting_Model = AdaBoostClassifier(random_state=1)
    adaBoosting_Model.fit(x_var, y_var)
    pprint.pprint(adaBoosting_Model)

    gradientboost_model = GradientBoostingClassifier(random_state=1)
    gradientboost_model.fit(x_var, y_var)
    pprint.pprint(gradientboost_model)

    models = {
        'Bagging Classifier'        : bagging_estimator,
        'RandomForest Model'        : rf_estimator,
        'Logistic Regression Model' : logistic_reg_model,
        'Decision Tree'             : dtree1,
        'Gradient Boost'            : gradientboost_model,
        'AdaBoostClassifier'        : adaBoosting_Model
    }
    
    return models

In [ ]:
def DisplayCoefficients(models, x_var):
    for model_name in models:        
        coefs = pd.DataFrame(
            np.append(models[model_name].coef_, models[model_name].intercept_),
            index=x_var.columns.tolist() + ["Intercept"],
            columns=["Coefficients"],
        )

        print(coefs.sort_values('Coefficients'))

In [ ]:
def GetModelsScoreDataFrame(models, x_train_var, x_test_var, y_train_var, y_test_var):
    scores = None
    for model_name in models:
        scores = GetMetricsScore(models[model_name], x_train_var, x_test_var, y_train_var, y_test_var)
        print(f"{model_name}")
    scores_overview_df = pd.DataFrame(columns=scores.keys())

    for model_name in models:
        scores = GetMetricsScore(models[model_name], x_train_var, x_test_var, y_train_var, y_test_var)
        scores_overview_df.loc[model_name] = scores

    return scores_overview_df

In [ ]:
models = BuildAndGetModels(x_train, y_train)
pprint.pprint(models)

In [ ]:
results = GetModelsScoreDataFrame(models, x_train, x_val, y_train, y_val)
results.head()

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 5. Model building - Oversampled data

In [ ]:
sm = SMOTE(sampling_strategy=0.4, k_neighbors=5, random_state=1)
x_train_over, y_train_over = sm.fit_resample(x_train, y_train)

def BuildAndGetModels(x_var, y_var):
    bagging_estimator=BaggingClassifier(random_state=1)
    bagging_estimator.fit(x_var, y_var)
    pprint.pprint(bagging_estimator)
    
    rf_estimator=RandomForestClassifier(random_state=1)
    rf_estimator.fit(x_var, y_var)
    pprint.pprint(rf_estimator)

    logistic_reg_model = LogisticRegression()
    logistic_reg_model.fit(x_var, y_var)
    pprint.pprint(logistic_reg_model)

    dtree1 = DecisionTreeClassifier(random_state=1)
    dtree1.fit(x_var, y_var)
    pprint.pprint(dtree1)

    adaBoosting_Model = AdaBoostClassifier(random_state=1)
    adaBoosting_Model.fit(x_var, y_var)
    pprint.pprint(adaBoosting_Model)

    gradientboost_model = GradientBoostingClassifier(random_state=1)
    gradientboost_model.fit(x_var, y_var)
    pprint.pprint(gradientboost_model)

    models = {
        'Bagging Classifier'        : bagging_estimator,
        'RandomForest Model'        : rf_estimator,
        'Logistic Regression Model' : logistic_reg_model,
        'Decision Tree'             : dtree1,
        'Gradient Boost'            : gradientboost_model,
        'AdaBoostClassifier'        : adaBoosting_Model
    }
    
    return models

In [ ]:
models = BuildAndGetModels(x_train_over, y_train_over)
pprint.pprint(models)

In [ ]:
results = GetModelsScoreDataFrame(models, x_train, x_val, y_train, y_val)
results.head()

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 6. Model building - Undersampled data

In [ ]:

rus = RandomUnderSampler(random_state=1, sampling_strategy = 1)
x_train_un, y_train_un = rus.fit_resample(x_train, y_train)

def BuildAndGetModels(x_var, y_var):
    bagging_estimator=BaggingClassifier(random_state=1)
    bagging_estimator.fit(x_var, y_var)
    pprint.pprint(bagging_estimator)
    
    rf_estimator=RandomForestClassifier(random_state=1)
    rf_estimator.fit(x_var, y_var)
    pprint.pprint(rf_estimator)

    logistic_reg_model = LogisticRegression()
    logistic_reg_model.fit(x_var, y_var)
    pprint.pprint(logistic_reg_model)

    dtree1 = DecisionTreeClassifier(random_state=1)
    dtree1.fit(x_var, y_var)
    pprint.pprint(dtree1)

    adaBoosting_Model = AdaBoostClassifier(random_state=1)
    adaBoosting_Model.fit(x_var, y_var)
    pprint.pprint(adaBoosting_Model)

    gradientboost_model = GradientBoostingClassifier(random_state=1)
    gradientboost_model.fit(x_var, y_var)
    pprint.pprint(gradientboost_model)

    models = {
        'Bagging Classifier'        : bagging_estimator,
        'RandomForest Model'        : rf_estimator,
        'Logistic Regression Model' : logistic_reg_model,
        'Decision Tree'             : dtree1,
        'Gradient Boost'            : gradientboost_model,
        'AdaBoostClassifier'        : adaBoosting_Model
    }
    
    return models

In [ ]:
models = BuildAndGetModels(x_train_un, y_train_un)
pprint.pprint(models)

In [ ]:
results = GetModelsScoreDataFrame(models, x_train, x_val, y_train, y_val)
results.head()

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 7. Hyperparameter tuning using random search

In [ ]:
bagging_estimator=BaggingClassifier(random_state=1)
bagging_estimator.fit(x_train, y_train)
pprint.pprint(bagging_estimator)

parameters = {
    'max_samples'  : [0.7, 0.8, 0.9, 1],
    'max_features' : [0.7, 0.8, 0.9, 1],
    'n_estimators' : [10, 20, 30, 40, 50]
}

acc_scorer = metrics.make_scorer(metrics.recall_score)
scorer = metrics.make_scorer(metrics.recall_score)
grid_obj = RandomizedSearchCV(
    estimator=bagging_estimator,
    param_distributions=parameters,
    n_jobs=-1,
    n_iter=20,
    scoring=scorer,
    cv=5,
    random_state=1,
)

gird_obj = grid_obj.fit(x_train_over,y_train_over)
bagging_estimator = grid_obj.best_estimator_
bagging_estimator.fit(x_train_over,y_train_over)

In [ ]:
rf_estimator=RandomForestClassifier(random_state=1)
rf_estimator.fit(x_train, y_train)
pprint.pprint(rf_estimator)

parameters = {
    'class_weight'      : [{0: 0.3, 1: 0.7}],
    'min_samples_leaf'  : np.arange(5, 10),
    'max_features'      : np.arange(0.2,0.7, 0.1),
    'max_samples'       : np.arange(0.3, 0.7, 0.1),
    "n_estimators"      : [100, 150, 200, 250]
}

acc_scorer = metrics.make_scorer(metrics.recall_score)
scorer = metrics.make_scorer(metrics.recall_score)
grid_obj = RandomizedSearchCV(
    estimator=rf_estimator,
    param_distributions=parameters,
    n_jobs=-1,
    n_iter=20,
    scoring=scorer,
    cv=5,
    random_state=1,
)

gird_obj = grid_obj.fit(x_train_over,y_train_over)
rf_estimator = grid_obj.best_estimator_
rf_estimator.fit(x_train_over,y_train_over)

In [ ]:
dtree1 = DecisionTreeClassifier(random_state=1)
dtree1.fit(x_train, y_train)
pprint.pprint(dtree1)

In [ ]:
adaBoosting_Model = AdaBoostClassifier(random_state=1)
adaBoosting_Model.fit(x_train, y_train)
pprint.pprint(adaBoosting_Model)

parameters = {
    "n_estimators": np.arange(10, 110, 10),
    "learning_rate": [0.1, 0.01, 0.2, 0.05, 1],
    "base_estimator": [
        DecisionTreeClassifier(max_depth=1, random_state=1),
        DecisionTreeClassifier(max_depth=2, random_state=1),
        DecisionTreeClassifier(max_depth=3, random_state=1),
    ],
}

acc_scorer = metrics.make_scorer(metrics.recall_score)
scorer = metrics.make_scorer(metrics.recall_score)
grid_obj = RandomizedSearchCV(
    estimator=adaBoosting_Model,
    param_distributions=parameters,
    n_jobs=-1,
    n_iter=20,
    scoring=scorer,
    cv=5,
    random_state=1,
)

gird_obj = grid_obj.fit(x_train_over,y_train_over)
adaBoosting_Model = grid_obj.best_estimator_
adaBoosting_Model.fit(x_train_over,y_train_over)

In [ ]:
gradientboost_model = GradientBoostingClassifier(random_state=1)
gradientboost_model.fit(x_train, y_train)
pprint.pprint(gradientboost_model)

parameters = {
    "subsample"       : [0.8, 0.9, 1],
    "n_estimators"    : [100, 150, 250],
    "max_features"    : [0.7, 0.8, 0.9, 1]
}


acc_scorer = metrics.make_scorer(metrics.recall_score)
scorer = metrics.make_scorer(metrics.recall_score)
grid_obj = RandomizedSearchCV(
    estimator=gradientboost_model,
    param_distributions=parameters,
    n_jobs=-1,
    n_iter=20,
    scoring=scorer,
    cv=5,
    random_state=1,
)

gird_obj = grid_obj.fit(x_train_over,y_train_over)
gradientboost_model = grid_obj.best_estimator_
gradientboost_model.fit(x_train_over,y_train_over)

In [ ]:
models = {
    'Bagging Classifier'        : bagging_estimator,
    'RandomForest Model'        : rf_estimator,
    'Decision Tree'             : dtree1,
    'Gradient Boost'            : gradientboost_model,
    'AdaBoostClassifier'        : adaBoosting_Model
}

In [ ]:
results = GetModelsScoreDataFrame(models, x_train, x_val, y_train, y_val)
results.head()

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 9. Pipeline

### Custom Transformation Classes

In [ ]:
'''
Input:
Target type and a list of feature names.

Output:
Convert all features provided in 'column_names' to Target type provided in 'toType'

Returns:
modifies main original data frame and returns nothing.
'''      
class ConvertObjectToCategoryType(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        catgry_col_names = x_dataset.select_dtypes(include=['object']).columns.tolist()
        for col_name in catgry_col_names:
            x_dataset[col_name] = x_dataset[col_name].astype("category")
        
        return x_dataset
    
class FixOutliers(BaseEstimator):

    def __init__(self, y):
        self.y = y
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        columns = x_dataset.select_dtypes(include=['float', 'int64']).columns.tolist()
        for column in columns:
            x_dataset[column] = BestCorrOutlierTreatment(column, TARGET_COLUMN, pd.concat([x_dataset, pd.Series(self.y, name=TARGET_COLUMN)], axis=1).reset_index())
        return x_dataset
    
class ReplaceMissingCatgrValues(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        x_dataset['Education_Level'].fillna(x_dataset['Education_Level'].value_counts().idxmax(), inplace=True)
        x_dataset['Marital_Status'].fillna(x_dataset['Marital_Status'].value_counts().idxmax(), inplace=True)
        return x_dataset
    
class ReplaceFeatureValuesWithIntValues(BaseEstimator):

    def __init__(self, y):
        self.y = y
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        replace_struct = {
            "Card_Category"       : {"Blue": 1, "Silver": 2, "good": 3, "Gold": 4, "Platinum": 5},
            "Income_Category"     : {"abc": -1, "Less than $40K": 1, "$40K - $60K": 2, "$60K - $80K": 3, "$80K - $120K": 4, "$120K +": 5},
            "Education_Level"     : {"Uneducated": 1, "High School": 2, "College": 3, "Graduate": 4, "Post-Graduate":5, "Doctorate": 6},
        }
        x_dataset =  x_dataset.replace(replace_struct)
        return x_dataset
    
class DropColumns(BaseEstimator):

    def __init__(self, y):
        self.y = y
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        df = pd.concat([x_dataset, self.y], axis=1).reset_index()
        list_of_features_to_drop = df.corr()[abs(df.corr()[TARGET_COLUMN]) < 0.05].index.to_list()
        df.drop(list_of_features_to_drop, axis=1, inplace=True)
        x_dataset =  df.drop([TARGET_COLUMN], axis = 1)
        return x_dataset
    
class DummyClassifier(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        return x_dataset

### Build The pipeline

In [ ]:
rus = RandomUnderSampler(random_state=1, sampling_strategy = 1)
x_train_un, y_train_un = rus.fit_resample(x_test, y_test)

#### Build Pipeline on Training Data

In [ ]:
pipeline = Pipeline([
    ('Fix_Columns_Types', ConvertObjectToCategoryType()),
    ('Missing_Values', ReplaceMissingCatgrValues()),
    ('Column_Transformation', ReplaceFeatureValuesWithIntValues(y_test)),
    ('OneHotEncoding', ce.OneHotEncoder(cols=["Gender", "Marital_Status"], use_cat_names=True)),
    ('Outlier_Treatment', FixOutliers(y_train_un)),
    ('make_target_int', ConvertObjectToCategoryType()),
    ('Drop_Features', DropColumns(y_train_un)),
    ('clsf', RandomForestClassifier(random_state=1))
])

In [ ]:
pipeline.fit(x_train_un, y_train_un)
pipeline.score(x_train_un, y_train_un)

#### Build Pipeline on Test Data

In [ ]:
pipeline = Pipeline([
    ('Fix_Columns_Types', ConvertObjectToCategoryType()),
    ('Missing_Values', ReplaceMissingCatgrValues()),
    ('Column_Transformation', ReplaceFeatureValuesWithIntValues(y_test)),
    ('OneHotEncoding', ce.OneHotEncoder(cols=["Gender", "Marital_Status"], use_cat_names=True)),
    ('Outlier_Treatment', FixOutliers(y_test)),
    ('make_target_int', ConvertObjectToCategoryType()),
    ('Drop_Features', DropColumns(y_test)),
    ('clsf', RandomForestClassifier(random_state=1))
])

In [ ]:
pipeline.fit(x_test, y_test)
pipeline.score(x_test, y_test)

####  <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> 10. Recommendations


- Attrition_Flag, Avg_Utilization_Ratio, Contacts_Count_12_mon, Months_Inactive_12_mon, Total_Ct_Chng_Q4_Q1, Total_Relationship_Count, Total_Revolving_Bal, Total_Trans_Amt, Total_Trans_Ct, these are the most important features to predict whether a customer will abondon the service or not.
- The number of clients is higher in customers who are Graduates
- The number of clients is higher in customers who are Married and Single
- The number of clients is higher in customers who earn less that 40k per year
- Blue card has the highest number of clients and seems to be the one brining in more money
- The charts sho a higher number of clients who are married and single
- Also the ratio of clients who left the service based on marital statis seems insignificance. Cannot deduce from the current of the data who category is most likely to stay or leave.
- Another noticeable difference is how dominant maried category is.
- Also most of the customers seems to be having a 2-3 dependents 
- The charts sho a higher number of clients who are married and single
- Also the ratio of clients who left the service based on marital statis seems insignificance. Cannot deduce from the current of the data who category is most likely to stay or leave.
- Another noticeable difference is how dominant maried category is.
- Also most of the customers seems to be having a 2-3 dependents 
- Again we see blue as the most dominant card in all categories.
- This blue card is also the one with the highest number of attrited customers.
- There is also a higher number of customers using the blue card who are earning less than 40k per year.
- We notice a spike or peak values in customers who are earning less than 40k per year
- The number of people leaving the credit card service based on income seems level or average accross all categories except for customers who are earning less than 40k per year with the highest number of attrition
- Customers who earn less than  40k per year also have 2-3 dependants
- The RandomForestClaffier model perfoms better (recall and F1 score) than all other models in most of the labs ran.
- We select the RandomForest model to create the pipeline.
- Suggestions are:
    - Look for programs to improve Avg_Utilization_Ratio
    - The lower the Total_Revolving_Bal, the more chances of increeasing attrition
    - decrease in Total_Trans_Ct means increase in retension, look for ways to help customers better use their cards less.